In [1]:
import re
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import KeyedVectors
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('stopwords')

# Load the text file
with open('data.txt', 'r', encoding='utf-8') as file:
    text = file.read()

# Split the text into paragraphs
paragraphs = text.split('\n\n')  # Assuming paragraphs are separated by double newlines

# Preprocess text
stop_words = set(stopwords.words('turkish'))

def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    words = word_tokenize(text)
    words = [word for word in words if word not in stop_words]
    return words

def preprocess_documents(documents):
    return [' '.join(preprocess(doc)) for doc in documents]

all_processed_docs = preprocess_documents(paragraphs)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\BatuhanYILMAZ\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\BatuhanYILMAZ\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# Load pre-trained Word2Vec model
word_vectors = KeyedVectors.load_word2vec_format('trmodel', binary=True)

# Function to expand query using Word2Vec
def expand_query(query, top_n=2):
    processed_query = preprocess(query)
    expanded_query = processed_query.copy()
    for word in processed_query:
        if word in word_vectors.key_to_index:
            similar_words = word_vectors.most_similar(positive=[word], topn=top_n)
            expanded_query.extend([word[0] for word in similar_words])
    return ' '.join(expanded_query)


In [3]:
# TF-IDF Vectorizer
cv = CountVectorizer()
word_count_vector = cv.fit_transform(all_processed_docs)
tfidf_transformer = TfidfTransformer(smooth_idf=True, norm='l2', use_idf=True)
tfidf_transformer.fit(word_count_vector)
tf_idf_vector = tfidf_transformer.transform(word_count_vector)
W = tf_idf_vector.toarray()


In [6]:
import numpy as np
# Function to respond to query
def respond(query, n_best=5):
    # Expand the query using Word2Vec
    expanded_query = expand_query(query)
    print(f"Expanded query: {expanded_query}\n")
    
    # Convert the expanded query to TF-IDF vector
    expanded_query_vec = cv.transform([expanded_query])
    tf_idf_query = tfidf_transformer.transform(expanded_query_vec)
    
    # Compute cosine similarity for TF-IDF
    tfidf_similarities = cosine_similarity(tf_idf_query, W)
    
    # Rank documents based on the TF-IDF similarities
    sorted_doc_indices = np.argsort(tfidf_similarities[0])[::-1]
    
    # Print the sorted documents and their similarity scores
    for idx in sorted_doc_indices[:n_best]:
        print(f"{paragraphs[idx]} with similarity score: {tfidf_similarities[0][idx]:.4f}\n")


query = 'yapay zeka ve sağlık'
respond(query)


Expanded query: yapay zeka sağlık suni zekâ zekâ zekanın saglık sağlik

Komplocu bunlar 1: Diyelim ki > bir ülkenin sağlık bakanlığı özel bir şirketin geliştirdiği yapay zeka destekli bir sağlık asistanını ve hastanesini akredite olarak tanımış olsun. Bu durumda bu kurum insanlara bir doktor yerine kendi sağlık asistanına tedavi olması gerektiğini ikna etmek için ne gibi değer önerileri sunabilirdi? with similarity score: 0.3665

İşte ütopik sağlık şirketimizin asistanına tedavi olmanı için önerilerimiz: 7/24 hızlı müdahale imkanı sağlar. Sürekli olarak en son tıbbi araştırmaları ve bulguları analiz eder, tüm branşların bilgisine sahip olup konsültasyon ihtiyacını ortadan kaldırır. İnsan faktöründen kaynaklanan hata oranını düşürür, büyük veri analizi ve geçmiş vakalardan elde edilen bilgilerle doğru teşhis ve tedavi imkanı sunar. Hastaların geçmiş sağlık verilerini kullanarak kişiselleştirilmiş tedavi planları oluşturur ve maliyetleri düşürerek daha fazla kişiye ulaşılabilir hale geti

In [7]:
query = 'risk altındaki meslekler'
respond(query)

Expanded query: risk altındaki meslekler riskler riskleri altında altındadır meslekleri rasyolar

Yapay zeka, hayatımıza girdiği günden beri özellikle kendi sektörümde bazı meslek gruplarının yaptığı işleri büyük oranda yapabildiğini izliyorum. Visual designer, concept artist, game artist, illustrator, copywriter, UX designer, UI designer ve daha niceleri…Yapay zekanın neleri değiştirebileceğini ve hangi meslekleri nasıl tehdit ettiğini evdeki koltuğunuzda sorgulamaya hazırsanız başlayalım… with similarity score: 0.1484

Yapay zekanın yargılamaları, insan hatasına daha az yer bırakarak daha adil ve tarafsız olma potansiyeline sahip. İstem dışı empati, kültürel kalıplar ve kişisel yönelimler gibi faktörlerin yargı sürecinden çıkarılmasıyla, daha objektif kararlar alınabilir gibi görünüyor. Bu, gerçekten işe yarayabilir mi? Belki de, yapay zeka sayesinde yargılama süreçleri daha güvenilir hale gelebilir. with similarity score: 0.0000

Hiçbir şey olmaz mı dersiniz ?
Bugün yeni GPT-4o’ya s

In [8]:
query = 'riskli meslek'
respond(query)

Expanded query: riskli meslek masraflı zahmetli mesleki hemşirelik

Yapay zeka, hayatımıza girdiği günden beri özellikle kendi sektörümde bazı meslek gruplarının yaptığı işleri büyük oranda yapabildiğini izliyorum. Visual designer, concept artist, game artist, illustrator, copywriter, UX designer, UI designer ve daha niceleri…Yapay zekanın neleri değiştirebileceğini ve hangi meslekleri nasıl tehdit ettiğini evdeki koltuğunuzda sorgulamaya hazırsanız başlayalım… with similarity score: 0.1484

Yapay zekanın yargılamaları, insan hatasına daha az yer bırakarak daha adil ve tarafsız olma potansiyeline sahip. İstem dışı empati, kültürel kalıplar ve kişisel yönelimler gibi faktörlerin yargı sürecinden çıkarılmasıyla, daha objektif kararlar alınabilir gibi görünüyor. Bu, gerçekten işe yarayabilir mi? Belki de, yapay zeka sayesinde yargılama süreçleri daha güvenilir hale gelebilir. with similarity score: 0.0000

Hiçbir şey olmaz mı dersiniz ?
Bugün yeni GPT-4o’ya sesli olarak bana logaritma öğr